# 学习率

在上一个章节，我们尝试使用梯度下降进行模型训练。结果一顿操作猛如虎，**损失值不仅没有减小，反而急剧增大**！

问题出在哪里？

答案是：**更新的步子迈得太大了！**

---

梯度的本质是损失函数对模型参数的导数，是损失函数曲线在某一点的切线斜率。它是下降的方向，并不是下降的幅度。

梯度很大，只代表道路很陡峭，不代表步子应该迈很大。

更新的步长太大，可能直接越过函数的最低点。后果就是损失值在最低点的左右震荡，无法收敛，最终导致模型训练失败。在深度学习中，这种现象被称为**发散**（Divergence）。

---

为控制更新的步长，我们引入一个比例系数，称为**学习率**（Learning Rate），记作 $\eta$。这样梯度下降的公式更新为：

$$
w_{\text{new}} = w_{\text{old}} - \eta \cdot \frac{\partial L}{\partial w}
$$
$$
b_{\text{new}} = b_{\text{old}} - \eta \cdot \frac{\partial L}{\partial b}
$$

通过调整学习率，我们可以控制模型训练的步幅。既不应太小，造成损失函数收敛太慢；也不能太大，造成损失函数左右震荡、甚至无法收敛。

## 超参数

学习率是我们遇到的第一个**超参数**（Hyperparameter）。

我们已经知道模型参数包括权重和偏置。模型参数是模型训练的对象；而超参数是我们控制模型训练幅度的参数，比如学习率是我们控制梯度下降步长的参数。

**超参数不会被模型学习**。而是需要在模型训练开始前，根据数据、经验等实际情况手动设定，并根据模型训练的效果进行调整，以期获得最佳训练结果。这个不断测试、调整超参数的过程，被称为**调参**（Hyperparameter Tuning）。

In [1]:
import numpy as np

## 数据

### 特征、标签

In [2]:
feature = np.array([28.1, 58.0])
label = np.array([165.0])

## 模型

### 参数：权重、偏置

In [3]:
weight = np.ones([1, 2]) / 2
bias = np.zeros(1)

### 推理函数

In [4]:
def forward(x, w, b):
    return x @ w.T + b

### 损失函数（平均平方差）

In [5]:
def mse_loss(p, y):
    return np.mean(np.square(y - p))

### 梯度函数

In [6]:
def gradient(p, y):
    return - 2 * (y - p)

### 反向函数

改进后的反向函数利用学习率来调整模型训练的幅度。

In [7]:
def backward(x, d, w, b, lr):
    w = w - d * x * lr
    b = b - d * lr
    return w, b

## 验证

### 超参数：学习率

学习率的选择依赖经验与实验。常见策略是从一个典型值（比如：0.1、0.01、0.001）开始，观察训练行为，逐步调整。

在我们的例子中，第一次模型训练得到的梯度值非常大。因此我们选择了一个较小的学习率:

In [8]:
LEARNING_RATE = 0.00001

### 推理

In [9]:
prediction = forward(feature, weight, bias)
print(f'prediction: {prediction}')

prediction: [43.05]


### 评估

In [10]:
loss = mse_loss(prediction, label)
print(f'loss: {loss}')

loss: 14871.802500000002


## 训练

### 梯度计算

In [11]:
delta = gradient(prediction, label)

### 反向传播

In [12]:
weight, bias = backward(feature, delta, weight, bias, LEARNING_RATE)
print(f"weight: {weight}")
print(f"bias: {bias}")

weight: [[0.5685359 0.641462 ]]
bias: [0.002439]


可以看出，通过引入合适的学习率，经过一次模型训练，权重和偏置的调整幅度变得相对合理。实际结果如何呢？

### 重新评估

In [13]:
prediction = forward(feature, weight, bias)
loss = mse_loss(prediction, label)
print(f'loss: {loss}')

loss: 12503.020514375934


通过重新评估可以看出：通过引入学习率，第一次模型训练使损失值下降了大约20%。这已经在一个正常、合理的范围了。

## 课后练习

尝试不同的学习率（如 0.001, 0.0001, 0.000001），观察：

* 权重和偏置的更新幅度；
* 损失值的变化趋势；
* 是否出现震荡或无变化的现象。